<a href="https://colab.research.google.com/github/FariaParvinMegha/thesis/blob/main/CNN%2BBiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
pip install git+https://github.com/forrestbao/pyeeg.git


  Cloning https://github.com/forrestbao/pyeeg.git to /tmp/pip-req-build-ac56nkii
  Running command git clone --filter=blob:none --quiet https://github.com/forrestbao/pyeeg.git /tmp/pip-req-build-ac56nkii
  Resolved https://github.com/forrestbao/pyeeg.git to commit a6c18bb093e4748f9d9c208535a6ae024a0802b8
  Preparing metadata (setup.py) ... done
  Created wheel for pyeeg: filename=pyeeg-0.4.4-py2.py3-none-any.whl size=28111 sha256=d422f73ea048ae8ff1552f22523061dd79f578e838640360f6634a57511a153c
  Stored in directory: /tmp/pip-ephem-wheel-cache-6hbncn8s/wheels/a8/c4/1a/cee09dcc12a11620066d35ace42e3c1e3bfbcc1db3a0ce7788
Successfully built pyeeg


In [7]:
import numpy as np
import pyeeg as pe
import pickle as pickle
import pandas as pd
import math
from sklearn.preprocessing import normalize

import os
import time

In [8]:
channel = [1,2,4,5,8,10,15,21,22,25,26,27,28,31,32]
band = [4,8,13,22,30,45] #5 bands
window_size = 256 #Averaging band power of 2 sec
step_size = 16 #Each 0.125 sec update once
sample_rate = 128 #Sampling rate of 128 Hz
subjectList = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32']


In [9]:

# Directory containing your data files
data_dir = '/content/drive/MyDrive/data_preprocessed_python'

# List of file names (assuming your files are named s01.dat, s02.dat, ..., s32.dat)
file_names = ['s{:02d}.dat'.format(i) for i in range(1, 33)]

for file_name in file_names:
    file_path = os.path.join(data_dir, file_name)

    with open(file_path, 'rb') as file:
        subject = pickle.load(file, encoding='latin1')

        eeg_data = []
        eeg_labels = []

        for i in range(0, 40):
            eeg_data.append(subject["data"][i])
            eeg_labels.append(subject["labels"][i])

        print("Processed:", file_name)
        print("EEG Data Shape:", np.array(eeg_data).shape)
        print("EEG Labels Shape:", np.array(eeg_labels).shape)

        # Modify the file name for saving
        save_name = file_name.replace('.dat', '_.npy')
        save_path = os.path.join(data_dir, save_name)

        np.save(save_path, eeg_data, allow_pickle=True, fix_imports=True)

        print("Saved:", save_name)
        print()



Processed: s01.dat
EEG Data Shape: (40, 40, 8064)
EEG Labels Shape: (40, 4)
Saved: s01_.npy

Processed: s02.dat
EEG Data Shape: (40, 40, 8064)
EEG Labels Shape: (40, 4)
Saved: s02_.npy

Processed: s03.dat
EEG Data Shape: (40, 40, 8064)
EEG Labels Shape: (40, 4)
Saved: s03_.npy

Processed: s04.dat
EEG Data Shape: (40, 40, 8064)
EEG Labels Shape: (40, 4)
Saved: s04_.npy

Processed: s05.dat
EEG Data Shape: (40, 40, 8064)
EEG Labels Shape: (40, 4)
Saved: s05_.npy

Processed: s06.dat
EEG Data Shape: (40, 40, 8064)
EEG Labels Shape: (40, 4)
Saved: s06_.npy

Processed: s07.dat
EEG Data Shape: (40, 40, 8064)
EEG Labels Shape: (40, 4)
Saved: s07_.npy

Processed: s08.dat
EEG Data Shape: (40, 40, 8064)
EEG Labels Shape: (40, 4)
Saved: s08_.npy

Processed: s09.dat
EEG Data Shape: (40, 40, 8064)
EEG Labels Shape: (40, 4)
Saved: s09_.npy

Processed: s10.dat
EEG Data Shape: (40, 40, 8064)
EEG Labels Shape: (40, 4)
Saved: s10_.npy

Processed: s11.dat
EEG Data Shape: (40, 40, 8064)
EEG Labels Shape: (4

In [39]:
from sklearn.utils import shuffle
data_training = []
label_training = []
data_validation = []
label_validation = []
data_testing = []
label_testing = []

for subjects in subjectList:
  with open('/content/drive/MyDrive/data_preprocessed_python/s' + subjects + '_.npy', 'rb') as file:

      sub = np.load(file, allow_pickle=True)
      sub = shuffle(sub, random_state=42)
      #for i in range (0,sub.shape[0]):
      #print(sub.shape[0])
        #if i % 8 == 0:
          #print(sub[i][0])
          #data_testing.append(sub[i][0])
          #label_testing.append(sub[i][1])
        #else:
          #data_training.append(sub[i][0])
          #label_training.append(sub[i][1])


      num_samples = sub.shape[0]

      # Calculate the number of samples for each split
      num_train_samples = int(0.8 * num_samples)
      num_validation_samples = int(0.1 * num_samples)

      for i in range(num_samples):
          if i < num_train_samples:
              data_training.append(sub[i][0])
              label_training.append(sub[i][1])
          elif i < num_train_samples + num_validation_samples:
              data_validation.append(sub[i][0])
              label_validation.append(sub[i][1])
          else:
              data_testing.append(sub[i][0])
              label_testing.append(sub[i][1])

np.save('/content/drive/MyDrive/data_training/data_training', np.array(data_training), allow_pickle=True, fix_imports=True)
np.save('/content/drive/MyDrive/label_training/label_training', np.array(label_training), allow_pickle=True, fix_imports=True)
print("training dataset:", np.array(data_training).shape, np.array(label_training).shape)

np.save('/content/drive/MyDrive/data_validation/data_validation', np.array(data_validation), allow_pickle=True, fix_imports=True)
np.save('/content/drive/MyDrive/label_validation/label_validation', np.array(label_validation), allow_pickle=True, fix_imports=True)
print("validation dataset:", np.array(data_validation).shape, np.array(label_validation).shape)

np.save('/content/drive/MyDrive/data_testing/data_testing', np.array(data_testing), allow_pickle=True, fix_imports=True)
np.save('/content/drive/MyDrive/label_testing/label_testing', np.array(label_testing), allow_pickle=True, fix_imports=True)
print("testing dataset:", np.array(data_testing).shape, np.array(label_testing).shape)

training dataset: (1024, 8064) (1024, 8064)
validation dataset: (128, 8064) (128, 8064)
testing dataset: (128, 8064) (128, 8064)


In [13]:
import keras
import keras.backend as K
from keras.layers.convolutional import Conv1D
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Model
import timeit
from keras.layers.convolutional import MaxPooling1D, ZeroPadding1D
from keras.optimizers import SGD
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [87]:
with open('/content/drive/MyDrive/data_training/data_training.npy', 'rb') as fileTrain:
    X  = np.load(fileTrain)

with open('/content/drive/MyDrive/label_training/label_training.npy', 'rb') as fileTrainL:
    Y  = np.load(fileTrainL)


X = normalize(X)
Z = np.ravel(Y[:, [1]])

Arousal_Train = np.ravel(Y[:, [0]])
Valence_Train = np.ravel(Y[:, [1]])
Domain_Train = np.ravel(Y[:, [2]])
Like_Train = np.ravel(Y[:, [3]])

In [88]:
from keras.utils import to_categorical
y_train = to_categorical(Z)
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [89]:
x_train = np.array(X[:])

In [78]:
with open('/content/drive/MyDrive/data_validation/data_validation.npy', 'rb') as fileVal:
    J  = np.load(fileVal, allow_pickle=True)

with open('/content/drive/MyDrive/label_validation/label_validation.npy', 'rb') as fileValL:
    K  = np.load(fileValL, allow_pickle=True)


J = normalize(J)
S = np.ravel(K[:, [1]])

Arousal_Test = np.ravel(K[:, [0]])
Valence_Test = np.ravel(K[:, [1]])
Domain_Test = np.ravel(K[:, [2]])
Like_Test = np.ravel(K[:, [3]])

In [79]:
y_val = to_categorical(S)
y_val

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [80]:
x_val = np.array(J[:])

In [81]:
with open('/content/drive/MyDrive/data_testing/data_testing.npy', 'rb') as fileTest:
    M  = np.load(fileTest, allow_pickle=True)

with open('/content/drive/MyDrive/label_testing/label_testing.npy', 'rb') as fileTestL:
    N  = np.load(fileTestL, allow_pickle=True)


M = normalize(M)
L = np.ravel(N[:, [1]])

Arousal_Test = np.ravel(N[:, [0]])
Valence_Test = np.ravel(N[:, [1]])
Domain_Test = np.ravel(N[:, [2]])
Like_Test = np.ravel(N[:, [3]])

In [82]:
x_test = np.array(M[:])

In [83]:
print(N.shape)

(128, 8064)


In [84]:
print("L indices:", L)

L indices: [-3.95329127e+00 -7.24817947e+00 -2.61060455e+00  1.60022210e+00
 -3.47287200e+00 -5.25782883e+01  3.84582238e+01 -3.83820740e+01
  9.45556995e+00 -5.03097059e+00 -7.31120664e+00 -5.42835838e+01
  3.46487266e+00 -1.31871228e+02  1.88593763e+01  2.92354509e+01
  1.31848989e+00  3.75051203e+00 -1.43818369e+01  1.67252357e+01
  2.14600223e+01  1.43714364e+01  2.73629835e+00 -2.58708273e+01
  8.03422593e+00 -2.78479391e-01 -2.57613498e+01 -2.21130318e+00
 -2.04556068e+00 -3.80685913e+01 -4.76120090e+00 -5.93985573e+00
  1.00847204e+00 -1.04424890e+00 -2.39848148e+01 -5.93131962e+00
 -3.63987155e+01 -1.27684395e+02 -6.22746671e+01 -3.17629473e+01
 -3.53548628e+01 -4.52943885e+01  2.62403928e+01 -3.27397185e+00
  5.37706133e+00  1.81733888e+01  1.85040330e+00  7.10774339e+00
 -4.10700570e+00 -1.11167078e+01  3.64442842e+00 -2.21369596e+01
  1.11066176e+01 -1.81529828e+01 -4.61631690e+00  5.20605997e+00
 -4.04849407e+00  1.37407486e+00  4.38866386e+00 -2.48200000e+01
  3.58400921e+

In [85]:
min_label = min(L)
L = L - min_label
y_test = to_categorical(L)
y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [90]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.fit_transform(x_val)
x_test = scaler.fit_transform(x_test)

In [91]:
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1], 1)
x_val = x_val.reshape(x_val.shape[0],x_val.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1], 1)
x_train
x_test
x_val

array([[[-0.01396073],
        [-0.27366057],
        [-0.08313386],
        ...,
        [-0.19718446],
        [-0.01819731],
        [-0.33145548]],

       [[-0.68590024],
        [ 0.36644433],
        [ 1.10518969],
        ...,
        [-1.08888771],
        [-0.82182566],
        [ 0.25835985]],

       [[ 0.10161039],
        [-0.25542287],
        [-0.59354699],
        ...,
        [-1.03058172],
        [ 0.42735062],
        [ 2.01047666]],

       ...,

       [[ 0.58136058],
        [ 0.85062326],
        [-0.70069902],
        ...,
        [-0.08660299],
        [-0.05195905],
        [ 0.39220041]],

       [[-0.85321314],
        [ 0.64132331],
        [ 1.4443649 ],
        ...,
        [-0.06981828],
        [-1.3577575 ],
        [-1.97713091]],

       [[ 0.20093834],
        [-0.53287901],
        [-0.68725511],
        ...,
        [ 1.41619704],
        [-0.40536874],
        [-1.48666726]]])

In [132]:
x_train.shape


(1024, 8064, 1)

In [104]:
y_val.shape

(128, 692)

In [105]:
y_test.shape

(128, 904)

In [122]:
batch_size = 256
num_classes = 10
epochs = 50
input_shape=(x_train.shape[1], 1)

In [123]:
print(input_shape)

(8064, 1)


In [124]:
num_classes = y_train.shape[1]
print("Number of classes:", num_classes)

Number of classes: 2


In [125]:
from keras.layers import Convolution1D, ZeroPadding1D, MaxPooling1D, BatchNormalization, Activation, Dropout, Flatten, Dense,  Bidirectional, LSTM
from keras.regularizers import l2

In [126]:
from keras.callbacks import EarlyStopping

In [127]:
model = Sequential()
intput_shape=(x_train.shape[1], 1)
model.add(Conv1D(128, kernel_size=3,padding = 'same',activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=(2)))
#model.add(Conv1D(128,kernel_size=3,padding = 'same', activation='relu'))
#model.add(BatchNormalization())
#model.add(MaxPooling1D(pool_size=(2)))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(32, return_sequences=True)))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_5 (Conv1D)           (None, 8064, 128)         512       
                                                                 
 batch_normalization_5 (Batc  (None, 8064, 128)        512       
 hNormalization)                                                 
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 4032, 128)        0         
 1D)                                                             
                                                                 
 bidirectional_10 (Bidirecti  (None, 4032, 128)        98816     
 onal)                                                           
                                                                 
 bidirectional_11 (Bidirecti  (None, 4032, 64)         41216     
 onal)                                                

In [128]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

In [129]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [131]:
history = model.fit(x_train, y_train_encoded,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_val, y_val_encoded),
          callbacks=[early_stopping])

ValueError: ignored